PROBLEM 1: 

An image feature is simply interesting or meaningful areas in an image such as the corners of a house or the peaks of a mountain. They are the most useful areas in an image to give enough information to describe the image. An image feature vector is just the numerical representation of these image features placed in a vector.  Typically many of these interest points would have to be found using something like sift rather than manually looking through an image since some points are not as intuitive as say the corners of a house.

In [ ]:
#PROBLEM 2


In [ ]:
#PROBLEM 3

In [ ]:
#PROBLEM 4